In [1]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

api_key = '6bsrcsp3ix2xnftcvqrfn25ts'
app_token = 'DMmIYVQ7ZdYrIqspWe1xXifsP'
url = 'https://data.cityofnewyork.us/resource/qp3b-zxtp.json'

def get_total_rows():
     params = {
        '$select':'count(*)',
        '$$app_token': app_token
    }
     try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for 4xx/5xx status codes
        total_rows=int(response.json()[0]['count'])
        return total_rows
     except requests.exceptions.RequestException as e:
        print(f"Request exception occurred: {e}")
        return 0
     
total_rows_in_api=get_total_rows()
print(total_rows_in_api)



39656098


In [2]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

api_key = '6bsrcsp3ix2xnftcvqrfn25ts'
app_token = 'DMmIYVQ7ZdYrIqspWe1xXifsP'
url = 'https://data.cityofnewyork.us/resource/qp3b-zxtp.json'

# Function to fetch data from the API with offset and limit
def fetch_data(offset):
    limit = 1000  # Limit for each request
    params = {
        '$limit': limit,
        '$offset': offset,
        '$$app_token': app_token
    }
    print(f"Fetching data with offset {offset}")
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for 4xx/5xx status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Request exception occurred for offset {offset}: {e}")
        return []

# Determine the total number of records to fetch
total_records = 120000
limit_per_request = 1000  # Limit per API request
max_workers = 10  # Adjust based on system and API limits

# Create a set to store unique offsets
used_offsets = set()

# Generate unique random starting points for each worker
while len(used_offsets) < (total_records // limit_per_request):
    random_offset = random.randint(0, total_rows_in_api-limit_per_request)
    used_offsets.add(random_offset)

# Convert the set to a list
offsets = list(used_offsets)

# Using ThreadPoolExecutor to fetch data concurrently
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(fetch_data, offset): offset for offset in offsets}
    
    results = []
    for future in as_completed(futures):
        offset = futures[future]
        try:
            data = future.result()
            results.extend(data)
        except Exception as e:
            print(f"Exception occurred for offset {offset}: {e}")

# Convert collected data to pandas DataFrame
df = pd.DataFrame(results)


Fetching data with offset 18950144
Fetching data with offset 30713861
Fetching data with offset 22259717
Fetching data with offset 29173766
Fetching data with offset 2531334
Fetching data with offset 34125832
Fetching data with offset 31070218
Fetching data with offset 35262474
Fetching data with offset 14176269
Fetching data with offset 32526353
Fetching data with offset 37687315
Fetching data with offset 25810966
Fetching data with offset 14278680
Fetching data with offset 3524635
Fetching data with offset 10434589
Fetching data with offset 26302494
Fetching data with offset 7991327
Fetching data with offset 28274718
Fetching data with offset 26417185
Fetching data with offset 25352227
Fetching data with offset 24367140
Fetching data with offset 24807459
Fetching data with offset 29233189
Fetching data with offset 36638759
Fetching data with offset 33075240
Fetching data with offset 28977191
Fetching data with offset 19787820
Fetching data with offset 20484141
Fetching data with offs

In [3]:
df.shape[0]

1200000

In [4]:
df.columns

Index(['vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'ratecodeid', 'store_and_fwd_flag',
       'pulocationid', 'dolocationid', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [5]:
df.head(5)

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-02-01T19:50:49.000,2022-02-01T20:11:00.000,2.0,4.9,1.0,Y,238,107,2,17.5,3.0,0.5,0.0,0.0,0.3,21.3,2.5,0.0
1,2,2022-02-01T19:32:35.000,2022-02-01T19:42:05.000,1.0,1.81,1.0,N,163,236,1,8.5,1.0,0.5,2.56,0.0,0.3,15.36,2.5,0.0
2,2,2022-02-01T19:55:04.000,2022-02-01T20:00:27.000,1.0,1.0,1.0,N,140,263,1,6.0,1.0,0.5,2.06,0.0,0.3,12.36,2.5,0.0
3,2,2022-02-01T19:39:59.000,2022-02-01T19:52:17.000,1.0,1.56,1.0,N,107,164,1,9.5,1.0,0.5,4.14,0.0,0.3,17.94,2.5,0.0
4,2,2022-02-01T19:58:25.000,2022-02-01T20:15:02.000,1.0,3.99,1.0,N,100,41,1,15.5,1.0,0.5,4.95,0.0,0.3,24.75,2.5,0.0


In [6]:
yellow_taxi_df=df

In [7]:
#checking for duplicated rows
duplicate_rows = yellow_taxi_df[yellow_taxi_df.duplicated()]

#display duplicate rows
print("Duplicate Rows:")
print(duplicate_rows)

Duplicate Rows:
        vendorid     tpep_pickup_datetime    tpep_dropoff_datetime  \
237000         1  2022-07-13T22:47:11.000  2022-07-13T23:03:10.000   
237001         2  2022-07-13T22:14:07.000  2022-07-13T22:21:06.000   
237002         2  2022-07-13T22:17:02.000  2022-07-13T22:31:06.000   
237003         2  2022-07-13T22:35:00.000  2022-07-13T22:51:28.000   
237004         2  2022-07-13T22:01:16.000  2022-07-13T22:08:44.000   
...          ...                      ...                      ...   
1176292        2  2022-04-11T10:31:14.000  2022-04-11T10:38:18.000   
1176293        2  2022-04-11T10:41:39.000  2022-04-11T11:03:09.000   
1176294        2  2022-04-11T10:11:01.000  2022-04-11T10:20:21.000   
1176295        2  2022-04-11T10:26:30.000  2022-04-11T10:40:49.000   
1176296        1  2022-04-11T10:27:03.000  2022-04-11T10:59:00.000   

        passenger_count trip_distance ratecodeid store_and_fwd_flag  \
237000              0.0           3.0        1.0                  N   
2

In [8]:
yellow_taxi_df['tpep_dropoff_datetime'] = pd.to_datetime(yellow_taxi_df['tpep_dropoff_datetime'])

In [9]:
yellow_taxi_df['year']=yellow_taxi_df['tpep_dropoff_datetime'].dt.year
yellow_taxi_df['month']=yellow_taxi_df['tpep_dropoff_datetime'].dt.month
yellow_taxi_df['day']=yellow_taxi_df['tpep_dropoff_datetime'].dt.day
yellow_taxi_df['hour']=yellow_taxi_df['tpep_dropoff_datetime'].dt.hour


In [10]:
dates = pd.to_datetime({"year": yellow_taxi_df.year, "month": yellow_taxi_df.month, "day": yellow_taxi_df.day})

In [11]:
yellow_taxi_df["day_of_week"] = dates.dt.dayofweek

In [12]:
yellow_taxi_df["weekend"] = dates.dt.dayofweek > 4

NameError: name 'yellow_taxi_df' is not defined

In [13]:
yellow_taxi_df.dtypes

vendorid                         object
tpep_pickup_datetime             object
tpep_dropoff_datetime    datetime64[ns]
passenger_count                  object
trip_distance                    object
ratecodeid                       object
store_and_fwd_flag               object
pulocationid                     object
dolocationid                     object
payment_type                     object
fare_amount                      object
extra                            object
mta_tax                          object
tip_amount                       object
tolls_amount                     object
improvement_surcharge            object
total_amount                     object
congestion_surcharge             object
airport_fee                      object
year                              int32
month                             int32
day                               int32
hour                              int32
day_of_week                       int32
weekend                            bool


In [14]:
seasons = {
    1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Autumn', 10: 'Autumn',
    11: 'Autumn', 12: 'Winter'
}


In [15]:
yellow_taxi_df['season'] = yellow_taxi_df['month'].map(seasons)


In [16]:
category_columns= ['year','month','day','hour','day_of_week']
for x in category_columns:
    yellow_taxi_df[x]=yellow_taxi_df[x].astype(str)

    

In [17]:
#checking for duplicated rows
duplicate_rows = yellow_taxi_df[yellow_taxi_df.duplicated()]

#display duplicate rows
print("Duplicate Rows:")
print(duplicate_rows)

Duplicate Rows:
        vendorid     tpep_pickup_datetime tpep_dropoff_datetime  \
237000         1  2022-07-13T22:47:11.000   2022-07-13 23:03:10   
237001         2  2022-07-13T22:14:07.000   2022-07-13 22:21:06   
237002         2  2022-07-13T22:17:02.000   2022-07-13 22:31:06   
237003         2  2022-07-13T22:35:00.000   2022-07-13 22:51:28   
237004         2  2022-07-13T22:01:16.000   2022-07-13 22:08:44   
...          ...                      ...                   ...   
1176292        2  2022-04-11T10:31:14.000   2022-04-11 10:38:18   
1176293        2  2022-04-11T10:41:39.000   2022-04-11 11:03:09   
1176294        2  2022-04-11T10:11:01.000   2022-04-11 10:20:21   
1176295        2  2022-04-11T10:26:30.000   2022-04-11 10:40:49   
1176296        1  2022-04-11T10:27:03.000   2022-04-11 10:59:00   

        passenger_count trip_distance ratecodeid store_and_fwd_flag  \
237000              0.0           3.0        1.0                  N   
237001              1.0          1.55

In [18]:
#yellow_taxi_df=yellow_taxi_df.drop_duplicates()
#print("Shape of dropped dataFrame:", yellow_taxi_df.shape)
#this shows all rows and columns of the new dataframe with dropped duplicates


In [19]:
#yellow_taxi_df=yellow_taxi_df.drop_duplicates()
#print("Shape of dropped dataFrame:", yellow_taxi_df.shape)
#this shows all rows and columns of the new dataframe with dropped duplicates


In [20]:
yellow_taxi_df.isnull().sum()

vendorid                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          34000
trip_distance                0
ratecodeid               34000
store_and_fwd_flag       34000
pulocationid                 0
dolocationid                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     34000
airport_fee              34000
year                         0
month                        0
day                          0
hour                         0
day_of_week                  0
weekend                      0
season                       0
dtype: int64

In [21]:
yellow_taxi_df.to_csv('yellow_taxi1.csv', index=False)

#### Yellow Taxi

In [22]:
yellow_taxi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 26 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   vendorid               1200000 non-null  object        
 1   tpep_pickup_datetime   1200000 non-null  object        
 2   tpep_dropoff_datetime  1200000 non-null  datetime64[ns]
 3   passenger_count        1166000 non-null  object        
 4   trip_distance          1200000 non-null  object        
 5   ratecodeid             1166000 non-null  object        
 6   store_and_fwd_flag     1166000 non-null  object        
 7   pulocationid           1200000 non-null  object        
 8   dolocationid           1200000 non-null  object        
 9   payment_type           1200000 non-null  object        
 10  fare_amount            1200000 non-null  object        
 11  extra                  1200000 non-null  object        
 12  mta_tax                12000

In [23]:
constant_columns=[]
for column in yellow_taxi_df.columns:
    if yellow_taxi_df[column].nunique()==1:
        constant_columns.append(column)

print('Constant columns:',constant_columns)
#there are no constant columns

Constant columns: []


In [24]:
yellow_taxi_df.describe()

,tpep_dropoff_datetime
count,1200000
mean,2022-07-05 16:09:44.256129792
min,2002-10-21 09:30:09
25%,2022-04-03 13:24:51.750000128
50%,2022-07-01 12:52:22.500000
75%,2022-10-09 18:19:24.500000
max,2023-01-01 19:38:05


Visualizations

In [25]:
import matplotlib.pyplot as plt

In [26]:
month =yellow_taxi_df['month'].value_counts()
month



month
3     123002
4     119022
12    116187
10    108010
11    107803
6      95998
5      94977
8      93009
2      91993
9      91993
7      82996
1      75010
Name: count, dtype: int64

In [27]:
seasons =yellow_taxi_df['season'].value_counts()
seasons



season
Spring    337001
Autumn    307806
Winter    283190
Summer    272003
Name: count, dtype: int64